In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import scvelo as scv
import scanpy

import warnings
warnings.filterwarnings('ignore')

import dynamo as dyn

dyn.get_all_dependencies_version()

|-----> setting visualization default mode in dynamo. Your customized matplotlib settings might be overritten.


package,umap-learn,pynndescent,python-igraph,numdifftools,seaborn,statsmodels,numba,scikit-learn,dynamo-release,cvxopt,pandas,scipy,numpy,colorcet,networkx,pre-commit,loompy,openpyxl,matplotlib,get-version,tqdm,setuptools
version,0.5.3,0.5.7,0.9.11,0.9.40,0.11.2,0.13.2,0.55.1,1.0.2,1.1.0,1.3.0,1.4.2,1.7.3,1.21.5,2.0.6,2.7.1,2.19.0,3.0.7,3.0.9,3.5.1,3.5.4,4.64.0,61.2.0


In [3]:
adata = scanpy.read_h5ad('/data/groups/yhhuang/scNT/neuron_splicing_lite.h5ad')
print(adata)


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/data/groups/yhhuang/scNT/neuron_splicing_lite.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)